In all versions 3.x the narrative gets cleaned as before and products are cleaned once times by randomly removing etries with the three most common values for product. This run uses 10 different principle components. 

__Initialing the data__

In [1]:
import pandas as pd

df_selected1 = pd.read_csv("../balanced_data/corpus_balanced_cleaned_lemmatized_first_cut.csv", encoding="utf-8")

print("nulls in df_selected:", df_selected1["Consumer complaint narrative"].isnull().sum())
df_selected = df_selected1.dropna()

nulls in df_selected: 6


__Initializing the bag of words__

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

# vect (bag of words)
count_vect = CountVectorizer(
    stop_words="english",
    #ngram_range=(1,2), # bigrammen
    min_df=2, # only keep words that appear twice
    max_df=0.5 # appears max in 50% of documents
)

X_train_counts = count_vect.fit_transform(df_selected["Consumer complaint narrative"])

__Principal components are chosen__

In [3]:
from sklearn.decomposition import TruncatedSVD

tSVD = TruncatedSVD(n_components=20)

principal_components = tSVD.fit_transform(X_train_counts)
print(principal_components.shape)

(292470, 20)


In [4]:
print(principal_components)
principal_components_df = pd.DataFrame(principal_components)
print(principal_components_df)

[[ 4.4436618  -3.22158948  0.01544162 ...  0.32700115 -0.07692991
  -0.76372175]
 [ 3.81940424 -1.11034471  0.33362425 ...  0.74523038  0.63241457
  -0.67960178]
 [11.86264223  4.43891137  4.55389933 ...  6.42830429  7.26493109
   1.36533457]
 ...
 [ 7.78327111  3.84666439 -3.26283368 ...  1.39660272 -0.4320766
  -1.48161657]
 [ 1.06129817  0.45517096 -0.09379111 ...  0.69294993  0.35559242
   0.08833654]
 [ 0.93802156  0.23178941  0.3857611  ...  0.23983491 -0.06493713
   0.10098243]]
                0         1         2         3         4         5         6  \
0        4.443662 -3.221589  0.015442  0.403845  0.058640  0.570516 -0.702397   
1        3.819404 -1.110345  0.333624 -1.003077 -1.508118  1.678713  1.520807   
2       11.862642  4.438911  4.553899  0.519017  3.177669 -2.706760 -3.357040   
3        2.077610  1.348609  1.202290 -0.667158 -2.074931  0.829459  1.339132   
4        5.176562 -4.094452 -1.813853 -4.858363  1.600993  2.177034 -0.301388   
...           ...      

__One hot encoding of extra columns__

In [5]:
#One hot encoding
issue_df = pd.Categorical(df_selected['Issue'])

df_dummies = pd.get_dummies(issue_df, prefix = 'issue')

#X_train_counts_df = pd.DataFrame(X_train_counts)
#print("DF conversion done")

print(principal_components_df.shape)
print(df_dummies.shape)
df_concat = pd.concat([principal_components_df, df_dummies], axis = 1)
print(df_concat.shape)


(292470, 20)
(292470, 161)
(292470, 181)


__Split into train and test__

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df_concat, df_selected['Product'])
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(219352, 181)
(73118, 181)
(219352,)
(73118,)


__Application of linear SVM__

In [7]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report # do more stats

classifier = svm.LinearSVC()
classifier.fit(X_train, Y_train)

test_predictions = classifier.predict(X_test)
print(classification_report(test_predictions, Y_test))

C:\Users\l-Larsovic-l\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\l-Larsovic-l\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                                              precision    recall  f1-score   support

                                                     Bank account or service       1.00      1.00      1.00      3720
                                                 Checking or savings account       0.99      1.00      1.00      4643
                                                               Consumer Loan       0.95      0.76      0.85      2901
                                                                 Credit card       1.00      1.00      1.00      4796
                                                 Credit card or prepaid card       0.96      0.98      0.97      7704
                                                            Credit reporting       1.00      1.00      1.00      7949
Credit reporting, credit repair services, or other personal consumer reports       0.99      0.91      0.95     11552
                                                       